In [1]:
import csv
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from statistics import mean
from numpy.linalg import norm
from pathlib import Path

C:\Users\kevin\anaconda3\envs\chatgpt\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Human Prompts

In [2]:
human_prompt_files = [
    "../sentence_embeddings/data/amazonTurkDesPrompt1.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt2.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt3.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt4.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt5.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt6.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt7.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt8.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt9.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt10.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt11.csv",
    "../sentence_embeddings/data/amazonTurkDesPromptNA.csv",
]

human_prompts = []

for human_prompt in human_prompt_files:
    prompts = []
    with open(human_prompt, 'r') as file:
        csvreader = csv.reader(file)
        for row in csvreader:
            prompts.append(row[0])
    prompts = [i.replace('\n',' ') for i in prompts]
    human_prompts.append(prompts)

In [3]:
pd.DataFrame(human_prompts)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,ï»¿A jump rope that is also to be used as a ti...,A muscle stimulate band that can be worn over ...,"A simple, but strong band that has a hook to b...","A calf strengthener, similar to a finger stren...",A rubber ball that can be inflated / deflated ...,An x shaped resistance band that has 2 hand gr...,A mat that has electrosensitive pressure point...,A fold bike that could be used as a bike and a...,I think there should be a product that will he...,I am thinking of a portable exercise device.,...,A vest that makes you flex your chest muscles ...,A fold up tension devise that can fit into a s...,something very small that will allow you to wo...,A rowing type machine that can be folded up in...,This device is a dumbbell in which the weighte...,a small collapsible step that can be used to d...,How about an abdominal or muscle training MP3 ...,"Flexible, non-electric, ergonomic, compact","A collapsible, compact elliptical machine.",I am thinking of some sort of rubber band stro...
1,ï»¿A vest made of crystal quartz pieces connec...,Perhaps there could be something...a cord?...a...,a device which is attached to the joints so wh...,"When you blush, you put sensors on your cheeks...",People could wear kinetic bracelets and anklet...,solar equipment collects energy,A bicycle-powered television,have electrodes that can suction cup to our bo...,A battery that is charged by the energy from y...,A shoe that collects the energy generated when...,...,Im thinking of clothing that contians statice ...,I know skin conductance goes up when people ar...,A stationary bike with the wheel connected to ...,put pressure buttons in the bottoms of shoes a...,I am thinking of something simple like a watch...,sort of like roller blades that store the mech...,Energy from the motion of walking could be col...,A smart phone app that trackA smart phone app ...,"To collect energy from human motion, a station...",Power collecting shoes that joggers or top rac...
2,ï»¿with a candle that has marks on it,A clock or new redesigned clock that measures ...,waterfall clock tracking time by water ammount,Instead of relying on visual inputs these woul...,A small chart on your wrist,A giant hourglass,A clock that used binary to measure time.,Marking a piece of string with knots to measur...,A medication that lets you be so in tune with ...,a clock that acts more like a measure of the s...,...,"""Timeline"" is a flexible strip of LED lights e...",Digital self-drawn laser clock art. The design...,Use base 10 instead of base-6 numbers.,A block you put in the freezer and it turns bl...,We could divide time up into different color q...,The sun app. The sun app uses a satellite in ...,It would be the number of times my fan turned ...,A device that monitors the rotations and posit...,TIME IS INFINITE THEREFORE SOMETHING HAS TO BE...,light bulbs that dim as the sun goes down
3,"ï»¿""A tall cylindrical bottle","Like a party pop, this would be for celebratio...",a turn table which senses something underneath...,It would be a puffer kind of like a perfume bo...,Use the same container that weed killer comes ...,I thought of an automatic powdered sugar dispe...,Conveyor belt through a box that sprays the po...,A metal powder shaker with small circular hole...,An electrically powered sifter. Like a hand po...,A small long object that looks like a scanner....,...,A series of spinning rotors move several spray...,"This would be like a car wash, but instead of ...",a cannister that sprays and coats something of...,A small canister with a spigot at the top and ...,A container with a perforated lid to dispense ...,A small salt-shaker like item. This one has a ...,"A cup/tube cover, with multiple layers that ha...",I bake often at home and this prompt made me i...,A device that can be used while cooking and ba...,An automatic brush machine that could be adjus...
4,"ï»¿""A retractable gripper that has a central h...",A rotating shelf

## Load GPT Prompts

In [4]:
def get_all_files(directory, pattern):
    return [f for f in Path(directory).glob(pattern)]

In [5]:
gpt_files = get_all_files("../prompt_engineering/data/few_shot/", "*.csv")
len(gpt_files)

60

In [6]:
gpt_prompts = []

# for design_prompt_count in range(11):
for iteration_count in range(5):
    iteration_prompts = []
    # for iteration_count in range(5):
    for design_prompt_count in range(12):
        fail = False
        prompt = pd.read_csv(f"../prompt_engineering/data/few_shot/design_prompt_{design_prompt_count}_{iteration_count}.csv", sep=",") #, header=None)
        lst = []
        for d in prompt.values:
            lst.append(d[0])
        prompt = []
        for i in range(len(lst)):
            try:
                mod_statements = lst[i].replace(lst[i][:3], '')
                prompt.append(mod_statements)
            except AttributeError:
                fail = True
        if fail:
            print(f"Could not process design_prompt_{design_prompt_count}_{iteration_count}")

        iteration_prompts.append(prompt)
    gpt_prompts.append(iteration_prompts)

Could not process design_prompt_1_0
Could not process design_prompt_0_1
Could not process design_prompt_0_2
Could not process design_prompt_10_2
Could not process design_prompt_9_3
Could not process design_prompt_10_3
Could not process design_prompt_10_4


## Embed data

In [9]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [10]:
human_embeddings = [model.encode(prompt) for prompt in human_prompts]

gpt_embeddings_iterations = []
for iteration in gpt_prompts:
    gpt_embeddings_iterations.append([model.encode(prompt) for prompt in iteration])


# Similarity Metrics
## Nearest Generated Sample
Using cosine distance
Focus on design space coverage

In [15]:
from itertools import combinations

distances_all = []
std_all = []
distances_total = []

for embedding in gpt_embeddings_iterations:
    distances_prompt = []
    std_prompt = []
    for count in range(len(embedding)):
        distances_row = []
        iterate = list(range(len(np.array(embedding[count]))))
        max_distance = -1e9
        for combo in combinations(iterate, 2):
            # dot product
            distance = np.dot(human_embeddings[count][combo[0],:], embedding[count][combo[1],:])/(norm(human_embeddings[count][combo[0],:])*norm(embedding[count][combo[1],:]))
            if distance > max_distance:
                max_distance = distance
            distances_row.append(max_distance)
        distances_total.append(np.array(distances_row))
        distances_prompt.append(np.mean(distances_row))
        std_prompt.append(np.std(distances_row))

    distances_all.append(distances_prompt)
    std_all.append(std_prompt)


In [16]:
distances_pd = pd.DataFrame(distances_all, range(5)).transpose()
distances_pd.style.background_gradient()

,0,1,2,3,4
0,0.803382,0.831761,0.641473,0.777952,0.811662
1,0.670527,0.784065,0.778102,0.795430,0.685583
2,0.696181,0.780873,0.757348,0.802845,0.739251
3,0.713660,0.704148,0.783050,0.767813,0.763004
4,0.775471,0.818587,0.811613,0.785756,0.807854
5,0.807617,0.821751,0.817345,0.811116,0.812543
6,0.792017,0.803559,0.808472,0.706480,0.836358
7,0.826030,0.857639,0.842654,0.808053,0.877208
8,0.813626,0.835568,0.777360,0.618698,0.740768
9,0.872596,0.888945,0.786405,0.712135,0.839304


In [17]:
std_pd = pd.DataFrame(std_all, range(5)).transpose()
std_pd.style.background_gradient()

,0,1,2,3,4
0,0.033709,0.061220,0.041303,0.038826,0.075292
1,0.084583,0.056096,0.059795,0.054530,0.086689
2,0.093582,0.063402,0.060394,0.083755,0.080604
3,0.089102,0.058289,0.098911,0.083438,0.086966
4,0.018786,0.023004,0.026316,0.028154,0.042148
5,0.060289,0.038914,0.079761,0.049371,0.046246
6,0.029742,0.064382,0.027094,0.048294,0.068543
7,0.023168,0.055185,0.029087,0.034148,0.023843
8,0.080060,0.080772,0.089772,0.194060,0.084948
9,0.027643,0.041355,0.095034,0.035861,0.028813


In [18]:
# aggregated mean across the x-axis 
distances_pd.mean(axis=1)

0     0.773246
1     0.742741
2     0.755300
3     0.746335
4     0.799856
5     0.814074
6     0.789377
7     0.842317
8     0.757204
9     0.819877
10    0.797777
11    0.842495
dtype: float32

In [19]:
# aggregated standard deviation across the x-axis
std_pd.mean(axis=1)

0     0.050070
1     0.068339
2     0.076347
3     0.083341
4     0.027682
5     0.054916
6     0.047611
7     0.033086
8     0.105922
9     0.045741
10    0.041040
11    0.037918
dtype: float32

In [20]:
# find total mean and total std
all_distance = np.array(distances_total)
mean_collect = []
stddev_collect = []
collec_num = []
for i in range(all_distance.shape[0]):
    std_dist = np.std(all_distance[i])
    mu_dist = np.mean(all_distance[i])
    std_dist_collec = (all_distance[i].shape[0]-1)*std_dist**2
    stddev_collect.append(std_dist_collec)
    mu_dist_collec = all_distance[i].shape[0]*mu_dist
    mean_collect.append(mu_dist_collec)
    collec_num.append(all_distance[i].shape[0])

total_mu = np.sum(np.array(mean_collect))/np.sum(np.array(collec_num))
total_std = np.sqrt(np.sum(np.array(stddev_collect))/np.sum(np.array(collec_num)))
print(total_mu)
print(total_std)

0.8037953404943928
0.05772339766390319


C:\Users\kevin\AppData\Local\Temp\ipykernel_19812\1780730982.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all_distance = np.array(distances_total)


# Novelty/Diversity Metrics
## Convex hull hypervolume

In [21]:
from scipy.spatial import ConvexHull
from sklearn.decomposition import PCA
from tqdm import tqdm

In [22]:
np.array(human_embeddings).shape

(12, 100, 384)

In [23]:
np.array([human_embeddings] + gpt_embeddings_iterations).shape

C:\Users\kevin\AppData\Local\Temp\ipykernel_19812\2793241421.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array([human_embeddings] + gpt_embeddings_iterations).shape


(6, 12)

In [24]:
pca = PCA(n_components=3)

volumes_all = []
for prompts in tqdm([human_embeddings] + gpt_embeddings_iterations):
    volumes_prompts = []
    for prompt in tqdm(prompts, leave=False):
        embeddings = pca.fit_transform(prompt)
        convex_hull = ConvexHull(embeddings)
        volume_prompt = convex_hull.volume
        volumes_prompts.append(volume_prompt)
    volumes_all.append(volumes_prompts)

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:02<00:00,  2.01it/s]


In [25]:
convex_hull_volumes_df = pd.DataFrame(volumes_all, index=['human', 'iteration 1', 'iteration 2', 'iteration 3', 'iteration 4', 'iteration 5']).transpose()
convex_hull_volumes_df.style.background_gradient()

,human,iteration 1,iteration 2,iteration 3,iteration 4,iteration 5
0,0.249209,0.272238,0.185465,0.178628,0.144559,0.255028
1,0.243630,0.139101,0.136639,0.226110,0.115445,0.196254
2,0.321649,0.295191,0.132038,0.173456,0.191801,0.265660
3,0.317053,0.141965,0.190726,0.211800,0.139574,0.117120
4,0.338390,0.056472,0.349410,0.220953,0.220271,0.077827
5,0.261036,0.083188,0.123826,0.196066,0.105052,0.240811
6,0.330278,0.247635,0.245502,0.301906,0.265899,0.243827
7,0.276590,0.142680,0.064899,0.126344,0.294472,0.075083
8,0.327088,0.167231,0.373653,0.352700,0.132803,0.251590
9,0.306566,0.280266,0.091362,0.096985,0.112691,0.114206


In [26]:
convex_hull_volumes_df.mean(axis=0)

human          0.294032
iteration 1    0.186456
iteration 2    0.195887
iteration 3    0.207756
iteration 4    0.162059
iteration 5    0.169529
dtype: float64

In [27]:
convex_hull_volumes_df.mean(axis=1)

0     0.214188
1     0.176196
2     0.229966
3     0.186373
4     0.210554
5     0.168330
6     0.272508
7     0.163345
8     0.267511
9     0.167013
10    0.149055
11    0.226401
dtype: float64

In [28]:
convex_hull_volumes_df.std(axis=1)

0     0.051385
1     0.053052
2     0.075065
3     0.073077
4     0.124244
5     0.074634
6     0.035822
7     0.099294
8     0.100527
9     0.098659
10    0.089587
11    0.075831
dtype: float64